In [1]:
import numpy as np
%matplotlib notebook
import matplotlib.pyplot as plt
import ipywidgets as widgets
import serial
import time
from picoscope import ps5000a
import picosdk
from picosdk.discover import find_all_units
import serial.tools.list_ports as port_list
import chipwhisperer as cw

In [2]:
scopes = find_all_units()
for scope in scopes:
    print("Working with:")
    print(scope.info)
    scope.close()
ports = list(port_list.comports())
for p in ports:
    print (p)


Working with:
UnitInfo(driver=<picosdk.ps5000a.Ps5000alib object at 0x7f641784d490>, variant=b'5244D', serial=b'KU687/0175')
/dev/ttyS4 - n/a


**PicoScope setup**

In [ ]:
ps = ps5000a.PS5000a()
print("Found the following picoscope:")
print(ps.getAllUnitInfo())

# Since target runnning at 10 MHz and AES requires from trigger
obs_duration = 3.225E-6
# Sample at least 1260 points within that window
sampling_interval = obs_duration / 1260
# Configure timebase
(actualSamplingInterval, nSamples, maxSamples) = ps.setSamplingInterval(sampling_interval, obs_duration)
print("Nsamples : ", nSamples)
print("Sampling interval = %f us" % (actualSamplingInterval*nSamples*1E6))

# 50mV range on channel A, AC coupled, 20 MHz BW limit
ps.setChannel('A', 'AC', 0.05, 0.0, enabled=True, BWLimited=True)
# Channel B is trigger
ps.setChannel('B', 'DC', 10.0, 0.0, enabled=True)
ps.setSimpleTrigger('B', 2.0, 'Rising', timeout_ms=2000, enabled=True)

The built target looks and feels like an STM32 target, as far as the capture HW is concerned

In [ ]:
TARGET_PLATFORM = 'CW305_100t'
fpga_id = '100t'
platform = 'cw305'
# program target:
bitstream = r"/home/sca.user/Desktop/arm_softcore/AT426-r0p1-00rel0-1/AT426-BU-98000-r0p1-00rel0/hardware/CW305_DesignStart/CW305_DesignStart.bit"
ftarget = cw.target(None, cw.targets.CW305, bsfile=bitstream, force=True)
print(ftarget.is_programmed())
print(ftarget.get_fpga_buildtime())

In [ ]:
# set clock:
ftarget.vccint_set(1.0)
# we only need PLL1:
ftarget.pll.pll_enable_set(True)            # enable PLL chip
ftarget.pll.pll_outenable_set(False, 0)     # disable PLL 0
ftarget.pll.pll_outenable_set(True, 1)      # enable PLL 1 
ftarget.pll.pll_outenable_set(False, 2)     # disable PLL 2

# run at 10 MHz:
ftarget.pll.pll_outfreq_set(10E6, 1)

**Capture loop**

In [ ]:
def pico_capture():
    # Arm the picoscope
    ps.runBlock()
    time.sleep(0.05)
    # Trigger the encryption on Target
    target.fpga_write(target.REG_USER_LED, [0x01])
    target.usb_trigger_toggle()
    ps.waitReady()
    # Capture the trace 
    data = ps.getDataV('A', nSamples, returnOverflow=False)
    return data

## Convenience functions  ##

In [ ]:
# some convenience functions:
def reset_fpga():
    # resets the full CW305 FPGA
    ftarget.fpga_write(3, [1])
    ftarget.fpga_write(3, [0])

def reset_arm_target():
    # resets only the Arm DesignStart core within the CW305 FPGA
    ftarget.fpga_write(2, [1])
    ftarget.fpga_write(2, [0])
    
def use_fpga_pll():
    # The target clock goes through a PLL (MMCM) in the FPGA before getting to the Arm DesignStart core.
    # This PLL can clean up the clock and filter glitches.
    ftarget.fpga_write(1, [1])

def bypass_fpga_pll():
    # The target clock is connected directly to the Arm DesignStart core, bypassing the PLL.
    # This can make clock glitching more effective.
    ftarget.fpga_write(1, [0])